In [1]:
import os, sys, time
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets')
import importlib
import GOSTnet as gn
importlib.reload(gn)
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.wkt import loads
from shapely.geometry import Point, LineString, MultiLineString
import networkx as nx
import osmnx as ox

peartree version: 0.6.1 
networkx version: 2.3 
matplotlib version: 3.0.3 
osmnx version: 0.9 
peartree version: 0.6.1 
networkx version: 2.3 
matplotlib version: 3.0.3 
osmnx version: 0.9 


In [2]:
npath = r'C:\Users\charl\Documents\GOST\Yemen\YEM\Round 3\output'
npickle = 'YEM_processed.pickle'
bpath = r'C:\Users\charl\Documents\GOST\Yemen'
opath = os.path.join(bpath, 'origins')
dpath = os.path.join(bpath, 'facility_files')
wpath = r'C:\Users\charl\Documents\GOST\Yemen\YEM\Round 3'

### Loads saved graph from pickle
Also in this section we take the largest two subgraphs of the network (for the mainland and the island of Socotra respectively) and keep only these roads going forward

In [6]:
G = nx.read_gpickle(os.path.join(npath, npickle))

list_of_subgraphs = list(nx.strongly_connected_component_subgraphs(G))

G_2 = list_of_subgraphs[0]
list_of_small_subgraphs = list_of_subgraphs[1:]

node_bunch = []
edge_bunch = []

for subgraph in list_of_small_subgraphs:
    if subgraph.number_of_nodes() > 50 and subgraph.number_of_nodes() != G_2.number_of_nodes():
        for u, data in subgraph.nodes(data = True):
            node_bunch.append((u, data))

        for u, v, data in subgraph.edges(data = True):
            edge_bunch.append((u, v, data))
    
G_2.add_nodes_from(node_bunch)
G_2.add_edges_from(edge_bunch)

G = G_2
print(G.number_of_edges())
gn.save(G, 'G', wpath)

290862


### Network Salting
Here, we break up long edges of the graph into sections of length no more than 'thresh'.
We do this to significantly improve the accuracy of snapping points off-network on to the network at the closest point. The GOSTnets function 'salt_long_lines' automatically makes these changes to the network for us

In [22]:
G_salty = gn.salt_long_lines(G, 
                             'epsg:4326', 
                             'epsg:32638', 
                             thresh = 2000, 
                             factor = 1000, 
                             attr_list = ['infra_type','id','country','osm_id','Type'])
G_salty = nx.convert_node_labels_to_integers(G_salty)

print('check: salting process has left number of connected components unchanged')
print(len(list(nx.strongly_connected_component_subgraphs(G))),
      ' | ', 
      len(list(nx.strongly_connected_component_subgraphs(G_salty))))

Identified 6063 unique edge(s) longer than 2000. 
Beginning new node creation...
44286 new edges added and 12125 removed to bring total edges to 323023
16080 new nodes added to bring total nodes to 123976
check: salting process has left number of connected components unchanged
2  |  2


In [23]:
print('adding traverse time edge property...')
# define speed limit dictionary
speed_dict = {
    'residential':25,
    'unclassified':15,
    'track':15,
    'tertiary':40,
    'secondary':50,
    'primary':60,
    'trunk':50,
    'service':15,
    'road':15,
    'trunk_link':50,
    'secondary_link':50,
    'primary_link':60,
    'tertiary_link':40}

# add traverse time property into 'time' edge attribute 
G_salty_time = gn.convert_network_to_time(G_salty, 
                                          distance_tag = 'length', 
                                          road_col = 'infra_type', 
                                          graph_type = 'drive', 
                                          speed_dict = speed_dict, 
                                          walk_speed = 4,
                                          factor = 1000
                                         )
gn.example_edge(G_salty_time, 2)

gn.save(G_salty_time, 'G_salty_time', wpath)

adding traverse time edge property...
(0, 35636, {'Wkt': 'LINESTRING (44.2165745 15.3646484, 44.2167032 15.3659494, 44.2167203 15.3663506, 44.2167763 15.3673335, 44.2167973 15.3681592)', 'id': 24386, 'infra_type': 'secondary', 'osm_id': 108470243, 'country': 'YEM', 'key': 'edge_24386', 'length': 389.3715222365589, 'Type': 'legitimate', 'time': 28.03474960103224, 'mode': 'drive'})
(0, 74084, {'Wkt': 'LINESTRING (44.2167973 15.3681592, 44.2168027 15.3690043, 44.2168968 15.3701596)', 'id': 24387, 'infra_type': 'secondary', 'osm_id': 108470243, 'country': 'YEM', 'key': 'edge_24387', 'length': 221.74928659443168, 'Type': 'legitimate', 'time': 15.96594863479908, 'mode': 'drive'})


### Manual Addition of Missing Roads
In the specific Yemen context, the border cuts off one crucial road link in the desert towards the northernmost border. Here, we manually re-add this road back to the network

In [24]:
missed_edges = []

st_node = 21793
end_node = 114778
st_point = Point(G_salty_time.nodes()[st_node]['x'],G_salty_time.nodes()[st_node]['y'])
end_point = Point(G_salty_time.nodes()[end_node]['x'],G_salty_time.nodes()[end_node]['y'])
lin = LineString([st_point,end_point])
real_length = 115

data = {'Wkt':lin,
       'id':max(nx.get_edge_attributes(G_salty_time,'id').values())+1,
       'infra_type':'service',
       'country':'YEM',
       'key':'manual_edge_1',
       'length':real_length, 
       'Type':'manual_edge',
       'time':float((real_length / 15)),
       'mode':'drive'}
missed_edges.append((st_node, end_node, data))
missed_edges.append((end_node, st_node, data))

G_salty_time.add_edges_from(missed_edges)

G_salty = nx.convert_node_labels_to_integers(G_salty)

gn.save(G_salty_time, 'G_salty_time', wpath)

### Snap Destination Files to Network

In [36]:
dest_df

,FID,Functionality,YEHNP_PHCs,YEHNP_Hospitals,Name of_Health_Facility,Latitude,Longitude,Name of Governorate,Name of District,Type of Area,...,EPI (P21a) 2016,Integrated Outreach (P22) 2016,Under-5 clinic services (P23) 2016,Malnutrition services (P25) 2016,Antenatal Care (P422) 2016,Skilled birth attendant (P423) 2016,Basic Emergency Obsteteric Care (P424) 2016,Emergency and Elective Surgery (S14) 2016,Comprehensive Emergency Obstetric Care (S424) 2016,geometry


In [44]:
dfiles = ['HeRAMS 2018 April.csv']
wpath = r'C:\Users\charl\Documents\GOST\Yemen\graphtool'

for dfile in dfiles:
    # Read in
    dest_df = pd.read_csv(os.path.join(os.path.join(dpath, dfile)), encoding = "ISO-8859-1")
    
    # Ensure coordinates are floats
    dest_df.Longitude = dest_df.Longitude.astype(float)
    dest_df.Latitude = dest_df.Latitude.astype(float)
    
    # Drop entries with no coordinates    
    dest_df2 = dest_df.copy()
    dest_df2 = dest_df2.loc[(dest_df2.Longitude != 0)]
    dest_df2 = dest_df2.loc[(dest_df2.Longitude != None)]
    print(len(dest_df))
    dest_df2 = dest_df2.loc[(dest_df2.Longitude <= 60)]
    dest_df2 = dest_df2.loc[(dest_df2.Longitude >= 35)]
    dest_df2 = dest_df2.loc[(dest_df2.Latitude <= 30)]
    dest_df2 = dest_df2.loc[(dest_df2.Latitude >= 5)]
    dest_df = dest_df2
    print(len(dest_df))
    
    
    # Generate Geometries
    dest_df['geometry'] = list(zip(dest_df.Longitude, dest_df.Latitude))
    dest_df['geometry'] = dest_df['geometry'].apply(Point)
    dest_df = gpd.GeoDataFrame(dest_df, geometry = 'geometry', crs = {'init':'espg:4326'})
    
    # Perform snap
    time.ctime()
    start = time.time()
    df = gn.pandana_snap(G_salty_time, dest_df, 'epsg:4326','epsg:32638', add_dist_to_node_col = True)
    
    # Save to file
    df.to_csv(os.path.join(wpath, dfile.replace('.csv', '_snapped.csv')))
    df.to_csv(os.path.join(dpath, dfile.replace('.csv', '_snapped.csv')))

    print('time elapsed: %d seconds' % (time.time() - start))

5042
5042
4411
time elapsed: 12 seconds


### Snap Origin Points to Networks

In [11]:
ofile_A = r'origins_1km_2015.csv'
ofile_B = r'origins_1km_2018.csv'
ofiles = [ofile_A,ofile_B]
wpath = opath 

for ofile in ofiles:
    
    # Read in
    dest_df = pd.read_csv(os.path.join(os.path.join(opath, ofile)), encoding = "ISO-8859-1")
    dest_df['geometry'] = dest_df['geometry'].apply(loads)
    dest_df = gpd.GeoDataFrame(dest_df, geometry = 'geometry', crs = {'init':'espg:4326'})
    
    # Perform snap
    print('Beginning snap')
    
    time.ctime()
    start = time.time()
    df = gn.pandana_snap(G_salty_time, dest_df, 'epsg:4326','epsg:32638', add_dist_to_node_col = True)
    
    # Save to file
    df.to_csv(os.path.join(wpath, ofile.replace('.csv', '_snapped.csv')))
    compath = r'C:\Users\charl\Documents\GOST\Yemen\graphtool'
    df.to_csv(os.path.join(compath, ofile.replace('.csv', '_snapped.csv')))

    print('Time elapsed: %d seconds' % (time.time() - start))

Beginning snap
Time elapsed: 161 seconds
Beginning snap
Time elapsed: 164 seconds
